In [86]:
import pandas as pd
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
# pulled in dataframe
df = pd.read_csv('../../src/data/syriatel_customer_churn.csv')

In [3]:
# dropped area code and phone number. still looking into if area code should be included
df = df.drop(['area code', 'phone number'], axis = 1)

In [4]:
X = df.drop('churn', axis=1)
y = df['churn']

In [5]:
from imblearn.pipeline import make_pipeline, Pipeline

In [64]:
# train-test-split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [65]:
# Our first model should have accuracy above 86 percent.
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(X_train, y_train)
dummy_model.score(X_train, y_train)

0.8567426970788315

In [73]:
def One_hot(X):
    '''
    Returns data frame of X_train and X_test of catagorical columns one 
    hot encoded merged with continous columns. 
    Catagorical anything that is an object and continous is not object
    '''
    X_num = X.select_dtypes(exclude='object') # numerical cols
    X_cat = X.select_dtypes(include='object') # categorical cols
    
    ohe = OneHotEncoder(drop='first', sparse=False)
    ohe.fit(X_cat)

    X_catagory = pd.DataFrame(ohe.transform(X_cat),
                                      columns=ohe.get_feature_names(),
                                        index=X.index)
    X_ohe = X_num.merge(X_catagory,
                           left_index=True,
                          right_index=True)
    


    return X_ohe

In [74]:
X_train_ohe = One_hot(X_train)

In [75]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_ss = pd.DataFrame(ss.fit_transform(X_train_ohe), index= X_train_ohe.index, columns= X_train_ohe.columns)

In [57]:
# for reproducibility purposes
sm = SMOTE(random_state=42)


# SMOTE number of neighbors


X_resample, y_resampled = sm.fit_resample(X_train_ss, y_train)

In [58]:
y_resampled.value_counts()

True     2141
False    2141
Name: churn, dtype: int64

In [77]:
X_test_ohe = One_hot(X_test)

In [78]:
ss = StandardScaler()
X_test_ss = pd.DataFrame(ss.fit_transform(X_test_ohe), index= X_test_ohe.index, columns= X_test_ohe.columns)

In [88]:
lg1 = LogisticRegression(random_state=42)

lg1.fit(X_resample, y_resampled)
y_hat_log1 = lg1.predict(X_test_ss)

print(lg1.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_log1))
print(classification_report(y_test, y_hat_log1))

0.7781774580335732
[[554 155]
 [ 30  95]]
              precision    recall  f1-score   support

       False       0.95      0.78      0.86       709
        True       0.38      0.76      0.51       125

    accuracy                           0.78       834
   macro avg       0.66      0.77      0.68       834
weighted avg       0.86      0.78      0.80       834



In [89]:
forest = RandomForestClassifier()
forest.fit(X_resample, y_resampled)
y_hat_forest = lg1.predict(X_test_ss)

print(forest.score(X_test_ss, y_hat_forest))
print(confusion_matrix(y_test, y_hat_forest))
print(classification_report(y_test, y_hat_forest))

0.7661870503597122
[[554 155]
 [ 30  95]]
              precision    recall  f1-score   support

       False       0.95      0.78      0.86       709
        True       0.38      0.76      0.51       125

    accuracy                           0.78       834
   macro avg       0.66      0.77      0.68       834
weighted avg       0.86      0.78      0.80       834

